In [4]:
%pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.9 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from dotenv import load_dotenv 

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_PROJECT'] = "Multi Agent Basics"
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [15]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
generate = prompt | llm

In [16]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

The Little Prince, a novella written by Antoine de Saint-Exupéry, is a timeless tale that has captivated readers of all ages since its publication in 1943. The story of a young prince who travels from planet to planet, encountering various strange characters and learning valuable lessons along the way, has become a beloved classic around the world. In today's fast-paced and technology-driven world, The Little Prince is more relevant than ever in modern childhood, offering valuable insights and lessons that are essential for the development of young minds.

One of the most significant reasons The Little Prince remains relevant in modern childhood is its exploration of themes that are highly relevant to contemporary children. The novella touches on issues such as the importance of human connections, the fleeting nature of material possessions, and the need for emotional intelligence. These themes are particularly relevant in today's society, where children are increasingly exposed to the

#### Reflect

In [17]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a teacher grading an essay submission. Generate critique and recomendations for the user's submission."
         " Provide detailed recommendations, including requests for length, depth, style, etc.",
         ),
         MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [18]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Overall, your essay provides a good introduction to the relevance of The Little Prince in modern childhood. You effectively explore the themes of human connections, emotional intelligence, and creativity, and highlight the novella's ability to transcend cultural and linguistic barriers. However, there are some areas that could be improved to take your essay to the next level.

Strengths:

* Your introduction effectively sets the stage for the rest of the essay, providing a clear thesis statement and establishing the significance of The Little Prince.
* Your analysis of the themes is thorough and well-supported, and you provide specific examples from the novella to illustrate your points.
* Your conclusion effectively summarizes the main points and reiterates the importance of The Little Prince in modern childhood.

Weaknesses:

* The essay could benefit from more depth and complexity in its analysis. While you provide some good examples, the analysis is often superficial and could be e

#### Graph

In [19]:
from typing import Annotated, List, Sequence
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict

class State(TypedDict):
    messages: Annotated[list, add_messages]

async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}

async def reflection_node(state: State) -> State:
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    return {"messages": [HumanMessage(content=res.content)]}

builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")

def should_continue(state: State):
    if len(state["messages"]) > 6:
        return END
    return "reflect"

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)


In [20]:
config = {"configurable": {"thread_id": "1"}}

async for event in graph.astream({
    "messages": [
        HumanMessage(
            content="Generate an essay on the topicality of the Little Prince and its message in the moder world"
        )
    ]
}, config):
    print(event)
    print("-----")

{'generate': {'messages': [AIMessage(content="Here is a 5-paragraph essay on the topicality of The Little Prince and its message in the modern world:\n\nThe Little Prince, written by Antoine de Saint-Exupéry in 1943, is a timeless tale that continues to captivate readers of all ages with its poignant exploration of human relationships, love, and the fragility of human connection. Despite being written over seven decades ago, the novella remains remarkably topical in today's world, offering a powerful commentary on the most pressing issues of our time. One of the most striking aspects of The Little Prince is its exploration of the human condition, particularly in the context of modern society's increasing detachment and disconnection from one another.\n\nThe novella's central theme of the importance of human relationships is more relevant than ever in today's world, where technology has led to a proliferation of social media platforms and online communication. While these tools have und